In [22]:
#Imports
import numpy as np
import matplotlib.pyplot as plt 
import scipy
from astropy.table import Table, join
import illustris_python as il
from astropy.table import setdiff
import time as time

In [47]:
"""
100-1 Loading
"""
basePath = '/home/mattbel/Desktop/Research/BhuvSummer2020/Illustris-100-1/output'
snapshot_ind = 99

"""
100-3 Loading

basePath = '/home/mattbel/Desktop/Research/BhuvSummer2020/Illustris-100-3/ouput'
snapshot_ind = 99 
"""
#Header gives basic group info
header = il.groupcat.loadHeader(basePath, snapshot_ind)
#Scale Factor
scale_factor = header['Time']

#Load in the subhalo/halo tables
subhalo_grp = il.groupcat.loadSubhalos(basePath, snapshot_ind)
halo_grp = il.groupcat.loadHalos(basePath, snapshot_ind)

#Get the keys for each
subhalo_keys = (list(subhalo_grp.keys()))
halo_keys = (list(halo_grp.keys()))

print(subhalo_grp['count'])
print(halo_grp['count'])

4371211
6291349
10.20634150505066


In [48]:
useless_keys_subhalo = ['count', 'SubhaloBHMdot','SubhaloBfldDisk','SubhaloBfldHalo','SubhaloGasMetalFractionsHalfRad',
                'SubhaloGasMetalFractionsMaxRad', 'SubhaloGasMetallicity', 'SubhaloGasMetallicityHalfRad',
                'SubhaloGasMetalFractionsSfr', 'SubhaloGasMetalFractionsSfrWeighted',
               'SubhaloGasMetallicityMaxRad','SubhaloGasMetallicitySfr','SubhaloGasMetallicitySfrWeighted', 
               'SubhaloIDMostbound', 'SubhaloSFRinHalfRad','SubhaloSFRinMaxRad', 'SubhaloSFRinRad',
               'SubhaloStarMetalFractionsHalfRad','SubhaloStarMetalFractionsMaxRad']
for i in range(0, len(useless_keys_subhalo)):
    subhalo_keys.remove(useless_keys_subhalo[i])

[subhalo_grp.pop(key) for key in useless_keys_subhalo] 

useless_keys_halo = ['count','GroupBHMdot','GroupGasMetalFractions', 'GroupStarMetalFractions']
    
for i in range(0, len(useless_keys_halo)):
    halo_keys.remove(useless_keys_halo[i])

[halo_grp.pop(key) for key in useless_keys_halo] 

0.023657560348510742


In [61]:
subhalo_keys = (list(subhalo_grp.keys()))
halo_keys = (list(halo_grp.keys()))

subhalo_table = Table(list(subhalo_grp.values()), names=(subhalo_keys))
subhalo_table.add_column([x for x in range(len(subhalo_table))], name='Index' ,index=0)

halo_table = Table(list(halo_grp.values()), names=(halo_keys))
halo_table.add_column([x for x in range(len(halo_table))], name='SubhaloGrNr' ,index=0)

subhalo_table = join(subhalo_table, halo_table, keys='SubhaloGrNr', join_type='left')

subhalo_table.add_column(subhalo_table['GroupFirstSub'] == subhalo_table['Index'],name=str("IsCentral"))
subhalo_table.add_column(subhalo_table["SubhaloMassInHalfRadType"][:,4], name=str("StellarMassHalfRad"))
subhalo_table.add_column(subhalo_table["SubhaloMassType"][:,4], name=str("StellarMass"))

33.09524393081665


In [62]:
print(len(subhalo_table))

4371211


In [63]:
def cut_color_lines(table, min_gmr, max_gmr):
    mag = table['SubhaloStellarPhotometrics']
    gmr_cut_low = (mag[:,4]-mag[:,5]) > min_gmr
    gmr_cut_high = (mag[:,4]-mag[:,5])< max_gmr
    return  table[gmr_cut_low & gmr_cut_high]

def cut_color_squares(table, min_gmr, max_gmr, min_rmz, max_rmz):
    #0.46, 0.34 min_gmr and max_rmz for my original "blue"
    #0.66, 0.52 min_gmr and min_rmz for my original "red"
    mag = table['SubhaloStellarPhotometrics']
    gmr_cut_low = (mag[:,4]-mag[:,5]) > min_gmr
    gmr_cut_high = (mag[:,4]-mag[:,5])< max_gmr
    rmz_cut_low = (mag[:,5]-mag[:,7])> min_rmz
    rmz_cut_high = (mag[:,5]-mag[:,7]) < max_rmz
    return table[gmr_cut_low & gmr_cut_high & rmz_cut_low & rmz_cut_high]

def cut_mass_high(table, field, num):
    if(field == "SubhaloMassType" or field == "SubhaloMassInRadType"):
        subhalomass = np.array(np.log10(table[field][:,4]*1e10))
        mask = subhalomass > num
        return table[mask]
    else:
        subhalomass = np.array(np.log10(table[field]*1e10))
        mask = subhalomass > num
        return table[mask]

def cut_mass_low(table, field, num):
    if(field == "SubhaloMassType" or field == "SubhaloMassInRadType"):
        subhalomass = np.array(np.log10(table[field][:,4]*1e10))
        mask = subhalomass < num
        return table[mask]
    else:
        subhalomass = np.array(np.log10(table[field]*1e10))
        mask = subhalomass < num
        return table[mask]

In [64]:
print(len(subhalo_table))
subhalo_table = subhalo_table[subhalo_table['SubhaloFlag']]
subhalo_table = subhalo_table[subhalo_table['SubhaloMassType'][:,4]!=0]
print(len(subhalo_table))

all_galaxies = cut_mass_high(subhalo_table, "GroupMass", 13)
all_galaxies = cut_mass_high(all_galaxies, "SubhaloMassType", 8)

print(len(all_galaxies))
print('Masking step 1 - remove low-mass subhalos - complete \n')

centrals = all_galaxies[all_galaxies['IsCentral']]
satellites = all_galaxies[np.bitwise_not(all_galaxies['IsCentral'])]

field_galaxies = centrals[centrals['GroupNsubs']<=1]
field_galaxies = field_galaxies[field_galaxies['IsCentral']]

print("Central galaxies: " + str(len(centrals)))
print("Satellite galaxies: " + str(len(satellites)))
print("Field Galaxies: " + str(len(field_galaxies)))
print("Masking step 2 - separate all centrals, satellites, and centrals with satellites in same halo, complete \n")

4371211
327324
11073
Masking step 1 - remove low-mass subhalos - complete 

Central galaxies: 183
Satellite galaxies: 10890
Field Galaxies: 0
Masking step 2 - separate all centrals, satellites, and centrals with satellites in same halo, complete 



In [66]:
t0 = time.time()
"""
Go through all halos and add those within a certain distance.
"""
#Examine only galaxies mass greater than 10^8
to_add = cut_mass_high(subhalo_table, "SubhaloMassType", 8)
print(len(to_add))

to_add = setdiff(to_add, satellites, keys='Index')
print(len(to_add))

to_add = setdiff(to_add, centrals, keys='Index')
print(len(to_add))

#iterate through all central galaxies
for i in range(0, len(centrals)):
    
    #get center of mass location for current central
    gal_cm = centrals['SubhaloPos'][i]
    
    #radius in Mpc/h to cut at
    value =  centrals['Group_R_Mean200'][i]*5*0.001
    
    #Get basic info about the box bounds to prevent wrapping issues.
    boxsize = header['BoxSize'] #kpc/h
    box_bounds = boxsize*np.ones(3)

    #Create an array of distances from all subhalos in the data satisfying a certain criterion to gal_cm
    dev = to_add['SubhaloPos'] - gal_cm
    for ind, q in enumerate(dev.T):
        q = np.where(np.abs(q) > 0.5 * box_bounds[ind], box_bounds[ind]-np.abs(q), q)
        dev.T[ind] = q  
    r_subhalo = np.linalg.norm(dev, axis=1)*.001*scale_factor #Mpc/h
    
    #Cut here
    to_add_now = to_add[r_subhalo < value]
    
    group_n_subs = np.array(to_add['GroupNsubs'])
    
    if(len(to_add_now)!=0):
        for k in range(1,len(halo_keys)):
            to_add_now[halo_keys[k]] = [centrals[halo_keys[k]][i]]*len(to_add_now)
        for j in range(0, len(to_add_now)):
            satellites.add_row(to_add_now[j])
            to_add = to_add[to_add['Index']!=to_add_now['Index'][j]]
            
    if(len(to_add)==0):
        break
print(time.time()-t0)

43820
32930
32747
122.96966934204102


In [ ]:
"""
Define color splits as per Tara's slide 24
"""
centrals_red = cut_mass_high(cut_color_lines(centrals, 0.6, 1000), "StellarMass" ,10.2)
centrals_green = cut_mass_low(cut_color_lines(centrals, 0.6, 1000), "StellarMass" ,10.2)
centrals_blue = cut_color_lines(centrals, 0, 0.5)

satellites_red = cut_mass_high(cut_color_lines(satellites, 0.6, 1000), "StellarMass" ,10.2)
satellites_green = cut_mass_low(cut_color_lines(satellites, 0.6, 1000), "StellarMass" ,10.2)
satellites_blue = cut_color_lines(satellites, 0, 0.5)

field_galaxies_red = cut_mass_high(cut_color_lines(field_galaxies, 0.6, 1000), "StellarMass" ,10.2)
field_galaxies_green = cut_mass_low(cut_color_lines(field_galaxies, 0.6, 1000), "StellarMass" ,10.2)
field_galaxies_blue = cut_color_lines(field_galaxies, 0, 0.5)

print("Central galaxies red: " + str(len(centrals_red)))
print("Central galaxies green: " + str(len(centrals_green)))
print("Central galaxies blue: " + str(len(centrals_blue)))

print("Satellite galaxies red: " + str(len(satellites_red)))
print("Satellite galaxies green: " + str(len(satellites_green)))
print("Satellite galaxies blue: " + str(len(satellites_blue)))

print("Field Galaxies red: " + str(len(field_galaxies_red)))
print("Field Galaxies green: " + str(len(field_galaxies_green)))
print("Field Galaxies blue: " + str(len(field_galaxies_blue)))

print("Masking step 3 - color splits for each type of galaxy, complete \n")

In [ ]:
def plot_all_bands(table):
    subhalomass = np.array(np.log10(table["SubhaloMassInHalfRad"]*1e104371211))
    mask = subhalomass > 10.3
    st = table[mask]
    fig, ax = plt.subplots(4,2)
    fig.set_size_inches(16,20)
    plt.subplots_adjust(hspace = 0.5)
    band_names = ['U', 'B', 'V','K', 'g', 'r', 'i', 'z']
    for bands in range(8):
        mag = st['SubhaloStellarPhotometrics'][:,bands]
        ax[bands%4][bands//4].hist(mag, bins=50)
        ax[bands%4][bands//4].set_xlabel('magnitude in ' + band_names[bands] + " band", fontsize = 16 )
        ax[bands%4][bands//4].set_ylabel('count', fontsize = 16)
    plt.show()

In [ ]:
def mass_vs_gmr(table, table_name, mass_type):
    #Recreating Figure 3 from https://arxiv.org/pdf/1707.03395.pdf
    mag = table['SubhaloStellarPhotometrics'][:,4] - table['SubhaloStellarPhotometrics'][:,5]
    mass = np.array(np.log10(table[mass_type]*1e10))
    plt.ylabel("$\mathrm{g-r}$", fontsize = 16)
    plt.xlabel("Mass", fontsize = 16)
    plt.title(table_name, fontsize = 16)
    #heights, mass_bins, mag_bins = np.histogram2d(mass, mag, bins=50)
    #plt.contour(mass_bins[0:50],mag_bins[0:50], heights, colors=['red','orange','pink','green'])
    plt.hist2d(mass, mag, bins=50, cmap='viridis', cmin=0.1) #, range=[[9,12], [0,1]])
    plt.show()

In [ ]:
def formation_rate_vs_gmr(table, table_name):
    #table = table[table['SubhaloSFR']>0]
    mag = table['SubhaloStellarPhotometrics'][:,4] - table['SubhaloStellarPhotometrics'][:,5]
    plt.figure(figsize=(8,6))
    mass = table['SubhaloSFR']
    plt.xlabel("$\mathrm{g-r}$", fontsize = 14)
    plt.ylabel("Stellar Formation Rate: Msun/yr", fontsize = 14)
    plt.title(table_name, fontsize=16)
    plt.hist2d(mag, mass, bins=50, cmap='viridis', cmin=0.1) # range=[[0,0.9], [0,0.8]])
    plt.colorbar()
    plt.show()

In [ ]:
def metallicity_vs_gmr(table, table_name):
    table = table[table['SubhaloSFR']>0]
    mag = table['SubhaloStellarPhotometrics'][:,4] - table['SubhaloStellarPhotometrics'][:,5]
    plt.figure(figsize=(8,6))
    metallicity = table['SubhaloStarMetallicity']
    plt.xlabel("$\mathrm{g-r}$", fontsize = 14)
    plt.ylabel("Mass-weighted average metallicity", fontsize = 14)
    plt.title(table_name, fontsize=16)
    plt.hist2d(mag, metallicity, bins=50, cmap='viridis', range=[[0,0.9], [0,0.08]], cmin=0.1)
    plt.colorbar()
    plt.show()

In [ ]:
def formation_rate_vs_mass(table, table_name, mass_type):
    table = table[table['SubhaloSFR']>0]
    mag = table['SubhaloStellarPhotometrics'][:,4] - table['SubhaloStellarPhotometrics'][:,5]
    #table = table[mag<0.55] #mag cut 
    mass = np.array(np.log10(table[mass_type]*1e10))
    plt.figure(figsize=(8,6))
    sfr = table['SubhaloSFR']
    plt.xlabel("Mass", fontsize = 14)
    plt.ylabel("Stellar Formation Rate: Msun/yr", fontsize = 14)
    plt.title(table_name, fontsize=16)
    plt.hist2d(mass, sfr, bins=50, cmap='viridis', cmin=0.1, range=[[8,12.5],[0,10]])
    plt.colorbar()
    plt.show()

In [ ]:
def hydrogen_rate_vs_gmr(table, table_name):
    table = table[table['SubhaloStarMetalFractions'][:,0]>0]
    mag = table['SubhaloStellarPhotometrics'][:,4] - table['SubhaloStellarPhotometrics'][:,5]
    plt.figure(figsize=(8,6))
    plt.xlabel("$\mathrm{g-r}$", fontsize = 14)
    plt.ylabel("Hydrogen Abundance", fontsize = 14)
    plt.title(table_name, fontsize=16)
    plt.hist2d(mag, table['SubhaloStarMetalFractions'][:,0], bins=50, cmap='viridis', cmin=0.1) #, range=[[0,0.9], [0.6,0.9]])
    plt.colorbar()
    plt.show()

In [ ]:
def helium_rate_vs_gmr(table, table_name):
    table = table[table['SubhaloStarMetalFractions'][:,1]>0]
    mag = table['SubhaloStellarPhotometrics'][:,4] - table['SubhaloStellarPhotometrics'][:,5]
    plt.figure(figsize=(8,6))
    plt.xlabel("$\mathrm{g-r}$", fontsize = 14)
    plt.ylabel("Helium Abundance", fontsize = 14)
    plt.title(table_name, fontsize=16)
    plt.hist2d(mag, table['SubhaloStarMetalFractions'][:,1], bins=50, cmap='viridis', cmin=0.1) #, range=[[0,0.9], [0.6,0.9]])
    plt.colorbar()
    plt.show()

In [ ]:
def element_rate_vs_mass(table, table_name, mass_type):
    elements = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Mg', 'Si', 'Fe']
    fig, ax = plt.subplots(3,3)
    fig.set_size_inches(16,16)
    for i in range (0, len(elements)):
        mass = np.array(np.log10(table[mass_type]*1e10))
        ax[i//3][i%3].set_xlabel("Mass: " + str(mass_type), fontsize = 11)
        ax[i//3][i%3].set_ylabel(elements[i] + " Abundance", fontsize = 11)
        ax[i//3][i%3].set_title(elements[i] + " in " + table_name, fontsize=12)
        ax[i//3][i%3].hist2d(mass, table['SubhaloStarMetalFractions'][:,i], bins=50, cmap='viridis', cmin=0.1)

In [ ]:
def element_rate_vs_gmr(table, table_name):
    elements = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Mg', 'Si', 'Fe']
    for i in range (0, len(elements)):
        #table = table[table['SubhaloStarMetalFractions']>0]
        mag = table['SubhaloStellarPhotometrics'][:,4] - table['SubhaloStellarPhotometrics'][:,5]
        plt.figure(figsize=(6,4))
        plt.xlabel("$\mathrm{g-r}$", fontsize = 11)
        plt.ylabel(elements[i] + " Abundance", fontsize = 11)
        plt.title(elements[i] + " in " + table_name, fontsize=12)
        plt.hist2d(mag, table['SubhaloStarMetalFractions'][:,i], bins=50, cmap='viridis', cmin=0.1) 
        plt.colorbar()
        plt.show()

In [ ]:
def gmr_mass_bins_hist(table, table_name, mass_type):
    bins = [9.0,9.5,10.0,10.5,11.0,11.5,12.0]
    fig, ax = plt.subplots(2,3)
    fig.set_size_inches(18,11)
    for i in range(0,6):
        s1 = cut_mass_high(table, mass_type, bins[i])
        s2 = cut_mass_low(table, mass_type, bins[i+1])
        mask2 = np.array(np.log10(s1[mass_type]*1e10)) < bins[i+1]
        s2 = s1[mask2]
        mag = s2['SubhaloStellarPhotometrics'][:,4] - s2['SubhaloStellarPhotometrics'][:,5]
        ax[i//3][i%3].hist(mag, bins=50) #, density=True)
        ax[i//3][i%3].set_title("Mass bin: "+str(bins[i])+"-"+str(bins[i+1]) + " for table: " + table_name)
        ax[i//3][i%3].set_xlabel('magnitude in g-r band', fontsize = 16 )
        ax[i//3][i%3].set_ylabel('count', fontsize = 16)

In [ ]:
def color_color(mag, r1, r2, table_name):
    g_minus_r = mag[:,4]-mag[:,5]
    r_minus_z = mag[:,5]-mag[:,7]
    plt.figure(figsize=(8,6))
    #heights, rmz,gmr = np.histogram2d(r_minus_z,g_minus_r,bins=150)
    plt.hist2d(g_minus_r,r_minus_z, bins = 50, cmap='viridis', range=[[r1, 1], [r2, 1]], cmin = 0.1)
    plt.xlabel("$\mathrm{g-r}$", fontsize = 20)
    plt.ylabel("$\mathrm{r-z}$", fontsize = 20)
    plt.title("Color-Color plot for " + table_name, fontsize = 20)
    plt.colorbar()
    plt.grid(b=True, which='both', axis='both')

In [ ]:
def color_density(mag, band1, band2, band3, title):
    band_names = ['U', 'B', 'V','K', 'g', 'r', 'i', 'z']
    r = mag[:,band1]
    r_minus_z = mag[:,band2]-mag[:,band3]
    plt.figure(figsize=(8,6))
    plt.hist2d(r, r_minus_z, bins = 50, cmap='viridis', cmin=0.1)#, range=[[0, 0.7], [0, 0.8]])
    plt.ylabel(band_names[band2]+"-"+band_names[band3], fontsize = 20)
    plt.xlabel(band_names[band1], fontsize = 20)
    plt.title(title, fontsize = 20)
    plt.colorbar()

In [ ]:
def galaxy_dist_vs_mass(table, table_name, color, mass_type, mass_show):
    
    boxsize = header['BoxSize'] #kpc/h
    box_bounds = boxsize*np.ones(3)#*conv_fac
    dev = table['SubhaloPos']-table['GroupPos']
 
    for ind, q in enumerate(dev.T):
        q = np.where(np.abs(q) > 0.5 * box_bounds[ind], box_bounds[ind]-np.abs(q), q)
        dev.T[ind] = q
    r_subhalo = np.linalg.norm(dev, axis=1)*.001*scale_factor #Mpc/h
    
    mass = np.array(np.log10(table[mass_type]*1e10))
    if(mass_show):
        plt.figure(figsize=(8,6))
        plt.hist2d(mass, r_subhalo, bins=50, cmap='viridis', cmin=0.1, range=[[9,12.5],[0,4]])
        plt.xlabel(mass_type)
        plt.ylabel("Distance in Mpc/h")
        plt.title("Distance of "  + color + " " + table_name + " from Halo Center vs. Mass.")
        plt.colorbar()
        plt.show()
    
    else:
        plt.hist(r_subhalo, bins=50)
        plt.title("Histogram of " + color + " " + table_name + " distance from Halo Center")
        plt.xlabel("Distance in Mpc/h")
        plt.show()

In [ ]:
def rotation_curve_vs_mass(table, table_name, color, mass_type):
    plt.figure(figsize=(8,6))
    mass = (np.log10(table[mass_type]*1e10))
    plt.hist2d(mass,table['SubhaloVmax'], bins=50, cmap='viridis', cmin=0.1, range=[[9,12.5],[0,800]])
    plt.title("Mass vs. max of rotation curve for " + color + " " + table_name)
    plt.ylabel("Max of spherically-averaged rotation curve. (km/s)", fontsize=14)
    plt.xlabel("Mass")
    plt.show()

In [ ]:
def peculiar_velocity_vs_mass(table, table_name, color, mass_type):
    v=table['SubhaloVel']-table['GroupVel']/scale_factor
    velocity = np.sqrt((v[:,0])**2 + (v[:,1])**2 +(v[:,2])**2)
    mass = np.array(np.log10(table[mass_type]*1e10))
    plt.figure(figsize=(8,6))
    plt.hist2d(mass,velocity, bins=50, cmap='viridis', cmin=0.1, range=[[8,12.5],[0,3000]])
    plt.title("Mass vs. Peculiar velocity " + color + " " + table_name , fontsize=14)
    plt.ylabel("Peculiar velocity of the subhalo. (km/s)", fontsize=14)
    plt.xlabel(mass_type, fontsize=14)
    plt.colorbar()
    plt.show()

In [ ]:
def phase_space(table, table_name, color, mass_type):
    v=table['SubhaloVel']-table['GroupVel']/scale_factor
    boxsize = header['BoxSize'] #kpc/h
    box_bounds = boxsize*np.ones(3)#*conv_fac
    dev = table['SubhaloPos']-table['GroupPos']
 
    for ind, q in enumerate(dev.T):
        q = np.where(np.abs(q) > 0.5 * box_bounds[ind], box_bounds[ind]-np.abs(q), q)
        dev.T[ind] = q
    
    r_subhalo = np.linalg.norm(dev, axis=1)*.001*scale_factor #Mpc/h
    dev = dev/(np.linalg.norm(dev, axis=1).reshape(-1,1))
    v_rad = np.empty([len(dev)])
    
    for i in range(0, len(dev)):
        v_rad[i]= np.dot(v[i],dev[i])

    plt.figure(figsize=(8,6))
    plt.hist2d(r_subhalo,v_rad, bins=30, cmap='viridis', cmin=0.1)
    plt.title("Phase Space " + color + " " + table_name , fontsize=14)
    plt.ylabel("Radial velocity of the subhalo. (km/s)", fontsize=14)
    plt.xlabel("Radius (Mpc/h)", fontsize=14)
    plt.colorbar()
    plt.show()
    
def tangential_v1(table, table_name, color, mass_type):
    v=table['SubhaloVel']-table['GroupVel']/scale_factor
    boxsize = header['BoxSize'] #kpc/h
    box_bounds = boxsize*np.ones(3)#*conv_fac
    dev = table['SubhaloPos']-table['GroupPos']
 
    for ind, q in enumerate(dev.T):
        q = np.where(np.abs(q) > 0.5 * box_bounds[ind], box_bounds[ind]-np.abs(q), q)
        dev.T[ind] = q
    
    r_subhalo = np.linalg.norm(dev, axis=1)*.001*scale_factor #Mpc/h
    dev = dev
    v_tan = np.empty([len(dev)])
    
    for i in range(0, len(dev)):
        cross_prod = np.cross(v[i],dev[i])
        norm_cross = np.linalg.norm(cross_prod)
        v_tan[i]= norm_cross/(np.linalg.norm(dev[i]))

    plt.figure(figsize=(8,6))
    plt.hist2d(r_subhalo,v_tan, bins=30, cmap='viridis', cmin=0.1)
    plt.title("Phase Space " + color + " " + table_name , fontsize=14)
    plt.ylabel("Tangential velocity of the subhalo. (km/s)", fontsize=14)
    plt.xlabel("Radius (Mpc/h)", fontsize=14)
    plt.colorbar()
    plt.show()

In [ ]:
def cent_acc_vs_rad(table, table_name, color, mass_type):
    v=table['SubhaloVel']-table['GroupVel']/scale_factor
    boxsize = header['BoxSize'] #kpc/h
    box_bounds = boxsize*np.ones(3)#*conv_fac
    dev = table['SubhaloPos']-table['GroupPos']
 
    for ind, q in enumerate(dev.T):
        q = np.where(np.abs(q) > 0.5 * box_bounds[ind], box_bounds[ind]-np.abs(q), q)
        dev.T[ind] = q
    
    r_subhalo = np.linalg.norm(dev, axis=1)*.001*scale_factor #Mpc
    dev = np.linalg.norm(dev, axis=1)
    v_sq = (np.linalg.norm(v, axis=1))**2
    rad_acc = v_sq/dev
    plt.figure(figsize=(8,6))
    plt.hist2d(r_subhalo,rad_acc, bins=30, cmap='viridis', cmin=0.1, range=[[0,8],[0,2000]])
    plt.title("Phase Space " + color + " " + table_name , fontsize=14)
    plt.ylabel("Radial acceleration of the subhalo. (km/s)", fontsize=14)
    plt.xlabel("Radius (Mpc/h)", fontsize=14)
    plt.colorbar()
    plt.show()

In [ ]:
arrs = [satellites_red, satellites_green, satellites_blue]
color = ['red', 'green', 'blue']
for i in range(0,3):
    boxsize = header['BoxSize'] #kpc/h
    box_bounds = boxsize*np.ones(3)#*conv_fac
    dev = arrs[i]['SubhaloPos']-arrs[i]['GroupPos']

    for ind, q in enumerate(dev.T):
        q = np.where(np.abs(q) > 0.5 * box_bounds[ind], box_bounds[ind]-np.abs(q), q)
        dev.T[ind] = q
    r_subhalo = np.linalg.norm(dev, axis=1)*.001*scale_factor #Mpc/h

    mass = np.array(np.log10(arrs[i]["StellarMass"]*1e10))
    plt.hist(r_subhalo, bins=50, color=color[i], histtype='step')# , density=True), ('label1', 'label2', 'label3'))
    print("Mean distance of " + color[i] + " galaxies from halo center in Mpc/h: " + str(np.mean(r_subhalo)))
    print("Number of subhalos that are " + color[i] + ": " +str(len(r_subhalo)) + "\n")
    
plt.title("Histogram of satellite galaxy distance from Halo Center by color.")
plt.xlabel("Distance in Mpc/h")
plt.show()

In [ ]:
#plot_all_bands(all_galaxies)
color_color(all_galaxies['SubhaloStellarPhotometrics'], 0.0, 0.0, "All Galaxies")
testing = cut_mass_high(all_galaxies, "StellarMass", 9)
color_color(testing['SubhaloStellarPhotometrics'], 0.0, 0.0, "Galaxies > 10^9 M_sun")
testing = cut_mass_high(all_galaxies, "StellarMass", 10)
color_color(testing['SubhaloStellarPhotometrics'], 0.0, 0.0, "Galaxies > 10^10 M_sun")
testing = cut_mass_high(all_galaxies, "StellarMass",11)
color_color(testing['SubhaloStellarPhotometrics'], 0.0, 0.0, "Galaxies > 10^11 M_sun")

In [ ]:
mass_vs_gmr(all_galaxies, "All Galaxies", "StellarMass")
mass_vs_gmr(centrals, "Central Galaxies", "StellarMass")
mass_vs_gmr(satellites, "Satellites", "StellarMass")
mass_vs_gmr(field_galaxies, "Field", "StellarMass")

In [ ]:
gmr_mass_bins_hist(all_galaxies, "All Galaxies" ,"StellarMass")
gmr_mass_bins_hist(satellites, "Satellite Galaxy" ,"StellarMass")
gmr_mass_bins_hist(centrals, "Centrals" ,"StellarMass")

In [ ]:
color_density(all_galaxies['SubhaloStellarPhotometrics'], 5, 4, 5, "Color vs. Magnitude Plot for all galaxies")
color_density(satellites['SubhaloStellarPhotometrics'], 5, 4, 5, "Color vs. Magnitude Plot for all satellites")
color_density(centrals['SubhaloStellarPhotometrics'], 5, 4, 5, "Color vs. Magnitude Plot for Centrals")
color_density(field_galaxies['SubhaloStellarPhotometrics'], 5, 4, 5, "Color vs. Magnitude Plot for field galaxies")

In [ ]:
testing = cut_mass_high(all_galaxies, "SubhaloMassInHalfRad", 9)
color_density(testing['SubhaloStellarPhotometrics'], 5,4, 5, "Color Magnitude for all galaxies masshalfrad>9")
testing = cut_mass_high(all_galaxies, "SubhaloMassInHalfRad", 9.5)
color_density(testing['SubhaloStellarPhotometrics'], 5,4, 5, "Color Magnitude for all galaxies masshalfrad>9.5")
testing = cut_mass_high(all_galaxies, "SubhaloMassInHalfRad", 10.5)
color_density(testing['SubhaloStellarPhotometrics'], 5,4, 5, "Color Magnitude for all galaxies masshalfrad>10.5")
testing = cut_mass_high(all_galaxies, "SubhaloMassInHalfRad", 11)
color_density(testing['SubhaloStellarPhotometrics'], 5,4, 5, "Color Magnitude for all galaxies masshalfrad>11")

In [ ]:
formation_rate_vs_gmr(all_galaxies, "All Galaxies")
formation_rate_vs_gmr(satellites, "Satellites")
formation_rate_vs_gmr(centrals, "Central Galaxies")
formation_rate_vs_gmr(field_galaxies, "Field Galaxies")

In [ ]:
testing = cut_mass_high(centrals_in_cluster, "SubhaloMassInHalfRad", 9.4)
formation_rate_vs_gmr(testing, "Central Galaxies in Clusters, mass>10^9.4")

In [ ]:
hydrogen_rate_vs_gmr(all_galaxies, "All Galaxies")
hydrogen_rate_vs_gmr(satellites, "Satellites")
hydrogen_rate_vs_gmr(centrals, "Central Galaxies")
hydrogen_rate_vs_gmr(field_galaxies, "Field Galaxies")

In [ ]:
testing = cut_mass_high(centrals_in_cluster, "SubhaloMassInHalfRad", 10)
hydrogen_rate_vs_gmr(testing, "Central Galaxies in Clusters, mass>10^10")

In [ ]:
helium_rate_vs_gmr(all_galaxies, "All Galaxies")
helium_rate_vs_gmr(satellites, "Satellites")
helium_rate_vs_gmr(centrals_in_cluster, "Central Galaxies in Clusters")
helium_rate_vs_gmr(field_galaxies, "Field Galaxies")

In [ ]:
mag = centrals_in_cluster['SubhaloStellarPhotometrics'][:,4] - centrals_in_cluster['SubhaloStellarPhotometrics'][:,5]
to_send = centrals_in_cluster[mag>0.55] 
element_rate_vs_mass(to_send, "Red Central Galaxies in Clusters", "StellarMass")
to_send = centrals_in_cluster[mag<0.55] 
element_rate_vs_mass(to_send, "Blue Central Galaxies in Clusters", "StellarMass")

In [ ]:
formation_rate_vs_mass(all_galaxies, "All Galaxies" ,"StellarMass")
formation_rate_vs_mass(centrals_in_cluster, "Central Galaxies in Clusters" ,"StellarMass")
formation_rate_vs_mass(satellites, "Satellites" ,"StellarMass")
formation_rate_vs_mass(field_galaxies, "Field Galaxies" ,"StellarMass")

In [ ]:
metallicity_vs_gmr(all_galaxies, "All Galaxies")
metallicity_vs_gmr(satellites, "Satellites")
metallicity_vs_gmr(centrals_in_cluster, "Central Galaxies in Clusters")
metallicity_vs_gmr(field_galaxies, "Field Galaxies")

In [ ]:
rotation_curve_vs_mass(satellites_red, "Satellite Galaxies", "red", "StellarMass")
rotation_curve_vs_mass(satellites_green, "Satellite Galaxies", "green", "StellarMass")
rotation_curve_vs_mass(satellites_blue, "Satellite Galaxies", "blue", "StellarMass")

rotation_curve_vs_mass(centrals_red, "Central Galaxies", "red", "StellarMass")
rotation_curve_vs_mass(centrals_green, "Central Galaxies", "green", "StellarMass")
rotation_curve_vs_mass(centrals_blue, "Central Galaxies", "blue", "StellarMass")

In [ ]:
peculiar_velocity_vs_mass(satellites_red, "Satellite Galaxies", "red", "StellarMass")
peculiar_velocity_vs_mass(satellites_green, "Satellite Galaxies", "green", "StellarMass")
peculiar_velocity_vs_mass(satellites_blue, "Satellite Galaxies", "blue", "StellarMass")

In [ ]:
phase_space(satellites, "Satellite Galaxies", "all", "StellarMass")
phase_space(satellites_red, "Satellite Galaxies", "red", "StellarMass")
phase_space(satellites_green, "Satellite Galaxies", "green", "StellarMass")
phase_space(satellites_blue, "Satellite Galaxies", "blue", "StellarMass")

In [ ]:
tangential_v2(satellites, "Satellite Galaxies", "all", "StellarMass")
tangential_v2(satellites_red, "Satellite Galaxies", "red", "StellarMass")
tangential_v2(satellites_green, "Satellite Galaxies", "green", "StellarMass")
tangential_v2(satellites_blue, "Satellite Galaxies", "blue", "StellarMass")

In [ ]:
tangential_v1(satellites, "Satellite Galaxies", "all", "StellarMass")
tangential_v1(satellites_red, "Satellite Galaxies", "red", "StellarMass")
tangential_v1(satellites_green, "Satellite Galaxies", "green", "StellarMass")
tangential_v1(satellites_blue, "Satellite Galaxies", "blue", "StellarMass")

In [ ]:
cent_acc_vs_rad(satellites, "Satellite Galaxies", "all", "StellarMass")
cent_acc_vs_rad(satellites_red, "Satellite Galaxies", "red", "StellarMass")
cent_acc_vs_rad(satellites_green, "Satellite Galaxies", "green", "StellarMass")
cent_acc_vs_rad(satellites_blue, "Satellite Galaxies", "blue", "StellarMass")